In [ ]:
from __future__ import  absolute_import
# though cupy is not used but without this line, it raise errors...
import cupy as cp
import os

import ipdb
import matplotlib.pyplot as plt
from tqdm import tqdm

from utils.config import opt
from data.dataset import Dataset, TestDataset, inverse_normalize
from model.faster_rcnn_vgg16 import FasterRCNNVGG16
from torch.utils import data as data_
import torch
from trainer import FasterRCNNTrainer
from utils import array_tool as at
from utils.vis_tool import vis_bbox
from utils.eval_tool import eval_detection_voc

# fix for ulimit
# https://github.com/pytorch/pytorch/issues/973#issuecomment-346405667
import resource

rlimit = resource.getrlimit(resource.RLIMIT_NOFILE)
resource.setrlimit(resource.RLIMIT_NOFILE, (2048, rlimit[1]))

In [ ]:
import requests

from tqdm import tqdm

def download_file_from_google_drive(id, destination):
    def get_confirm_token(response):
        for key, value in response.cookies.items():
            if key.startswith('download_warning'):
                return value

        return None

    def save_response_content(response, destination):
        CHUNK_SIZE = 32768

        with open(destination, "wb") as f:
            with tqdm(unit='B', unit_scale=True, unit_divisor=1024) as bar:
                for chunk in response.iter_content(CHUNK_SIZE):
                    if chunk:  # filter out keep-alive new chunks
                        f.write(chunk)
                        bar.update(CHUNK_SIZE)

    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)

# Download Models from Google Drive

In [ ]:
if not os.path.isdir("checkpoints"):
    os.mkdir("checkpoints")

# Download Baseline Model
download_file_from_google_drive("1GyttG9S55QsK3FuXqNof866OdHiwW7R6", "./checkpoints/fasterrcnn_set00")
# Download Pruned Network
download_file_from_google_drive("121OCJTAcFt-9l5XZeiFOIuxGOOHN9typ", "./checkpoints/fasterrcnn_set00pruned")
# Download SparseDense Network (WeightSharing Applied)
download_file_from_google_drive("1TrB631H3LNxHBRMAKIrpSCnVYIU_V-uL", "./checkpoints/sparsedense_fasterrcnn")


In [ ]:
def train(**kwargs):
    opt._parse(kwargs)
    return opt

# Input data directory and path to model
# Use Baseline Model
opt = train(voc_data_dir='dataset', load_path='./checkpoints/fasterrcnn_set00')

# Use Pruned Model
#opt = train(voc_data_dir='dataset', load_path='./checkpoints/fasterrcnn_set00pruned')

# Use SparseDense Model
# opt = train(voc_data_dir='dataset', load_path='./checkpoints/sparsedense_fasterrcnn')

In [ ]:
# User arguments
#print(opt._parse_all())

# Load dataset
dataset = TestDataset(opt, split='val')
dataloader = data_.DataLoader(dataset,
                                   batch_size=1,
                                   num_workers=opt.test_num_workers,
                                   shuffle=False,
                                   pin_memory=True
                                   )

# Construct model
faster_rcnn = FasterRCNNVGG16(mask=opt.mask)
trainer = FasterRCNNTrainer(faster_rcnn).cuda()
trainer.load(opt.load_path)

# Predict
imgs, sizes, gt_bboxes_, gt_labels_ = next(iter(dataloader))
sizes = [sizes[0][0].item(), sizes[1][0].item()]
pred_bboxes_, pred_labels_, pred_scores_ = faster_rcnn.predict(imgs,[sizes])

# Show score
ori_img_ = inverse_normalize(at.tonumpy(imgs[0]))
gt_img = vis_bbox(ori_img_,
                     at.tonumpy(gt_bboxes_[0]),
                     at.tonumpy(gt_labels_[0]))
plt.show()

pred_img = vis_bbox(ori_img_,
                    at.tonumpy(pred_bboxes_[0]),
                    at.tonumpy(pred_labels_[0]).reshape(-1),
                    at.tonumpy(pred_scores_[0])
                    )
plt.show()